# Plot OpenFAST Outputs

In [6]:
# Load all the tools
import numpy as np
# from wisdem.aeroelasticse.CaseLibrary import ROSCO_Test
import os, subprocess

from weis.aeroelasticse.FAST_reader import InputReader_Common, InputReader_OpenFAST, InputReader_FAST7
from weis.aeroelasticse.FAST_writer import InputWriter_Common, InputWriter_OpenFAST, InputWriter_FAST7
from weis.aeroelasticse.runFAST_pywrapper import runFAST_pywrapper_batch
from weis.aeroelasticse.CaseGen_General import CaseGen_General
from weis.aeroelasticse.CaseGen_IEC import CaseGen_IEC
from pCrunch import pdTools
from pCrunch import Processing, Analysis
from weis.aeroelasticse.Util import FileTools
# Instantiate fast_IO
from ROSCO_toolbox import utilities as ROSCO_utilities
fast_io = ROSCO_utilities.FAST_IO()
fast_pl = ROSCO_utilities.FAST_Plots()
import pandas as pd

import matplotlib.pyplot as plt
# %matplotlib

i_fig = 0

Set outputs

In [12]:
outfiles = [
    '/Users/dzalkind/Tools/WEIS-3/results/CT-spar/DISCON_CT-spar_lowBW/ntm+max_tq/iea15mw_055.outb',
#     '/Users/dzalkind/Tools/ROSCO_toolbox/Examples/examples_out/13_Simulink_Test/OL_Test_1.SFunc.outb',
#     '/Users/dzalkind/Tools/WEIS-3/sowfa_debug/rotor_sweep/c_001_sp6_h150_D240_oR2_yaw_base/IEA-15-240-RWT-Monopile.1.T1.out',
#     '/Users/dzalkind/Tools/WEIS-3/sowfa_debug/rotor_sweep/c_001_sp6_h150_D240_oR2_yaw_base/IEA-15-240-RWT-Monopile.2.T2.out',
#     '/Users/dzalkind/Tools/WEIS-3/results/CT-barge/DISCON-CT-barge_hiBW/simp/step_1.outb',
]



In [13]:
# pull from eagle if necessary
pull_script = '/Users/dzalkind/Tools/WEIS-3/pCrunch/HPC_tools/eagle2local_sim_inp.sh'


basedir = outfiles[0].split('/')[1]
# if basedir == 'scratch':
    
temp = outfiles[0].split('/')
filedir = '/'.join(temp[:-1]) + '/'

namebase = temp[-1].split('.')[0]
search_str = namebase + '*'

shell_cmd = pull_script + ' ' + filedir + ' ' + namebase

shell_cmd
# ret = subprocess.call(pull_script,shell=True)


'/Users/dzalkind/Tools/WEIS-3/pCrunch/HPC_tools/eagle2local_sim_inp.sh /Users/dzalkind/Tools/WEIS-3/results/CT-spar/DISCON_CT-spar_lowBW/ntm+max_tq/ iea15mw_055'

In [14]:
# ret

In [15]:
output_ext = '.out'
plt.rcParams["figure.figsize"] = [9,7]

#  Define Plot cases 
cases = {}
cases['Gen. Speed Sigs.'] = ['Wind1VelX', 'BldPitch1', 'GenTq', 'GenSpeed','TwrBsMyt','GenPwr']#,'PtfmPitch','PtfmYaw','NacYaw']
cases['Debug'] = ['IPDefl1','OoPDefl1','Azimuth','RotTorq']#,'PtfmPitch','PtfmYaw','NacYaw']
cases['Plt. Control Sigs.'] = ['RtVAvgxh', 'BldPitch1', 'Fl_Pitcom', 'PC_MinPit','WE_Vw']
cases['Platform Motion'] = ['PtfmSurge', 'PtfmSway', 'PtfmHeave', 'PtfmPitch','PtfmRoll','PtfmYaw']


# Rosco outfiles
r_outfiles = [out.split('.out')[0] + '.RO.dbg' for out in outfiles]

fast_out = fast_io.load_fast_out(outfiles, tmin=0)
rosco_out = fast_io.load_fast_out(r_outfiles, tmin=0)
  
rosco_out
# Combine outputs
comb_out = [None] * len(fast_out)
for i, (r_out, f_out) in enumerate(zip(rosco_out,fast_out)):
    r_out.update(f_out)
    comb_out[i] = r_out

# comb_out
# if False:
#     comb_out[0]['meta']['name'] = 'Variable Bandwidth'

    
# Plot
fig, ax = fast_pl.plot_fast_out(cases, comb_out, showplot=True)

if False:  # Print!
    
    save_fig_dir = '/Users/dzalkind/Projects/CarbonTrust/Deliverables'
    for f in fig:
        f.savefig(os.path.join(save_fig_dir,'ts'+str(i_fig)))
        i_fig += 1

RtVAvgxh is not available as an output channel.
Fl_Pitcom is not available as an output channel.
PC_MinPit is not available as an output channel.
WE_Vw is not available as an output channel.


In [16]:
# Plot Spectral

fastp = ROSCO_utilities.FAST_Plots()

fast_out

[{'Time': array([0.0000e+00, 1.0000e-02, 2.0000e-02, ..., 7.9998e+02, 7.9999e+02,
         8.0000e+02]),
  'Wind1VelX': array([11.09194235, 11.09123733, 11.09070857, ..., 10.98953822,
         11.04082841, 11.09194235]),
  'Wind1VelY': array([-0.1757434 , -0.23681083, -0.29771277, ..., -0.1227852 ,
         -0.1492643 , -0.1757434 ]),
  'Wind1VelZ': array([-0.81162075, -0.8355739 , -0.85952705, ..., -0.5618075 ,
         -0.68665764, -0.81162075]),
  'Azimuth': array([  0.        ,   0.45044613,   0.895399  , ..., 135.62273635,
         136.00726353, 136.39179071]),
  'BldPitch1': array([5.23383852, 5.21372564, 5.19377897, ..., 2.8118148 , 2.81364324,
         2.81563791]),
  'GenSpeed': array([7.54662762, 7.48071132, 7.41967189, ..., 6.40025048, 6.39969987,
         6.39922791]),
  'IPDefl1': array([ 1.43718377e-05, -3.73557228e-03, -1.43486217e-02, ...,
         -1.76861493e+00, -1.75842641e+00, -1.74823788e+00]),
  'LSSGagMya': array([12189.77071503, 12392.3506912 , 11812.30155079, 